╔══<i><b>Public-AI</b></i>═══════════════════════════════════════════════════╗
### ✎&nbsp;&nbsp;week 1. 고객 데이터와 개인화 추천시스템

# Section 1. 비개인화 추천 시스템

개인화 추천 시스템이 부상하기 전에도 추천시스템은 존재했습니다. 그 중 대표적인 방법이 '순위'를 이용한 비개인화 추천입니다. <br>
누가 어디서 무엇을 구매했느냐 등의 정보를 복합적으로 고려하는 개인화 추천시스템과 다르게 비개인화 추천은 주어진 데이터 안에서 인기도나 최신성 등의 통계치를 내어 정렬을 하고 상위의 상품을 노출합니다. <br>
<br>
이번에는 본격적으로 개인화 추천시스템을 배우기에 앞서 비개인화된 방식으로 추천할 아이템을 찾아봅시다. <br>
실무 환경에서 추천 시스템을 구성해보아야 하기 때문에 관계형 데이터베이스에 접속하여 데이터에 접근하고 데이터를 가져와보겠습니다.<br>
데이터베이스에 직접 쿼리하는 쿼리문을 작성해 기준에 따라 통계치를 계산하여 추천할 아이템을 찾아주는 시스템을 만들어봅시다. <br>


### _Objective_ 
* [데이터베이스의 데이터 확인하기] RDB에 연결하여 데이터를 확인하고, 필요에 따라 SQL 쿼리를 작성하여 데이터베이스로부터 데이터를 가져오는 방법을 배웁니다. 
* [데이터 탐색하기] Pandas를 이용해 데이터를 탐색하고, 원하는 형태에 따라 데이터를 변형하는 방법을 배웁니다. 
* [비개인화 추천: 아이템의 순위 매기기] 비개인화 추천시스템 중 대표적인 방법인 순위를 활용한 추천 방식을 배웁니다.

╚═══════════════════════════════════════════════════════════╝

In [4]:
# 필요한 라이브러리 가져오기
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.utils import get_file

# 영화 포스터를 가져와 주피터에서 볼 수 있도록 만들어주는 메소드
def display_posters(movieId_list):
    import requests
    from io import BytesIO
    from PIL import Image
    
    def get_poster(movieId):
        url = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/img/POSTER_20M_FULL/{}.jpg".format(movieId)
        try:
            response = requests.get(url)
            b = BytesIO(response.content)
            img = np.asarray(Image.open(b))
        except:
            img = np.zeros((200,100,3))
        return img
    
    images = []
    for i in movieId_list:
        img = get_poster(i)
        images.append(img)

    columns = 5
    rows = (len(movieId_list) - 1) // columns + 1
    fig, axes = plt.subplots(rows, columns)
    axes = axes.flatten()
    fig.set_size_inches((20,rows * 5))
        
    for ind, img in enumerate(images):
        c_id = ind % columns
        r_id = ind // columns
        ax = axes[r_id*columns + c_id]
        ax.set_title(movieId_list[ind])
        ax.imshow(img)

    plt.tight_layout()

# \[데이터베이스의 데이터 확인하기\]
---

## 1. 데이터베이스에 접속하기

많은 데이터베이스 프로그램은 클라이언트/서버 형태로 가동하고 있습니다. <br>
회사 전산망 혹은 클라우드에 데이터를 일괄적으로 저장하는 서버 컴퓨터가 별도로 존재하고, 해당 서버 컴퓨터에 접속 권한을 받아 들어가는 방식으로 동작합니다. <br>
보통 데이터베이스 서버를 접속하기 위해서는 아래의 정보들이 필요합니다. <br>

| 인자  | 설명 |
|------|-----|
| host | 데이터베이스 서버의 주소(URL)|
| db   | 데이터베이스 이름| 
| user | 사용자 이름 |
| password | 사용자 비밀번호 |

`pymysql`은 파이썬이 관계형 데이터베이스인 MariaDB와 연결할 수 있게 도와주는 엔진 패키지입니다.<br>
이 패키지를 통해 우리는 파이썬을 이용해 하나인데이터베이스로 데이터를 주고 받을 수 있게 됩니다. <br>

````python
import pymysql
# import MySQLdb # 빠르지만, 세팅하는데에 좀 어려움이 있어요 -> mysql-client
````

본 수업에서는 개별 컴퓨터에 관계형 데이터베이스를 구축하지 않고, 클라우드 서버를 이용합니다. <br>
아마존 클라우드 컴퓨팅 서비스 AWS(Amazon Web Services) 중 하나인 EC2를 이용해 클라우드 서버를 생성하고, MariaDB를 구축해두었습니다.<br>

In [5]:
import pymysql

connect = pymysql.connect(
    host='ec2-13-209-14-240.ap-northeast-2.compute.amazonaws.com',
    user='data-engineer', 
    password='datadata',
    db='movielens')

## 2. Movie-Lens 데이터베이스의 테이블 확인하기

데이터베이스에는 여러 개의 테이블이 존재합니다. `connect`에 연결된 데이터베이스의 이름은 `movielens`입니다. <br>
Movie-Lense 데이터는 크게 영화의 장르가 담긴 `genres` 데이터와 영화 정보가 담긴 `movies` 데이터, 영화별 평점이 담긴 `ratings` 데이터로 나뉩니다.<br>
세 가지 데이터를 각각의 테이블로 구성하여 `movielens` 데이터베이스에 있습니다. 

데이터베이스에 어떤 테이블이 있는지를 확인하기 위해서는 아래와 같은 쿼리를 이용합니다.

````sql
SHOW TABLES -- 데이터베이스 내 테이블 이름 보여주기
````

In [15]:
# 데이터 베이스 내 테이블이 무엇인지 찾기


구체적으로 테이블 내에 어떠한 필드(열)가 있는지를 확인하기 위해서는 `DESC` 명령어를 이용하면 됩니다.

````sql
    DESC 테이블 이름 --- 테이블 내 컬럼 정보 가져오기
````    

In [6]:
# (1) movies 테이블에 대한 컬럼 정보 가져오기


In [7]:
# (2) ratings 테이블에 대한 컬럼 정보 가져오기


In [8]:
# (3) genres 테이블에 대한 컬럼 정보 가져오기


## 3. SQL 구문을 통해 데이터 확인하기

데이터베이스에서 데이터를 전부 가져오는 작업은 데이터베이스에 부하를 주고 데이터를 가져와서 띄우는 컴퓨터에도 메모리 부담이 생기기 때문에 지양해야 합니다. <br>
필요한 데이터를 가져오기 전에 샘플 데이터를 보고 데이터의 형태를 확인하고, 필요한 데이터에 맞게 쿼리를 작성해야 합니다.<br>
<br>
테이블에서 원하는 데이터를 지정해서 가져오는 기본 문법은 아래와 같습니다.<br>
````sql
SELECT 열 이름 FROM 테이블 이름 LIMIT 가져올 데이터 수
```` 


### (1) `movies` 테이블 확인하기

위의 코드를 통해 우리는 테이블 내 레코드들을 가져올 수 있습니다.

In [9]:
# movies 테이블에서 10개 행만 가져오기


`movies`에는 `id` 칼럼과 제목에 해당하는 `title` 칼럼, 영화가 개봉된 연도인 `release_year`가 있습니다. <br>
이 테이블에서 `id`와 `title`만 가져오고 싶다면 아래와 같이 작성하면 됩니다.

In [10]:
# movies 테이블의 id, title 열에 대한 10개 행 가져오기



### (2) `genres` 테이블 확인하기

이번에는 장르 데이터를 살펴봅시다.

In [12]:
# genres 테이블에서 10개 행만 가져오기


`genres` 테이블은 `movies` 테이블의 `id`에 해당하는 `movie_id` 칼럼이 있고, 해당 영화의 장르를 나타내는 `genres` 칼럼이 있습니다. <br>
데이터를 보면, `movie_id` 1번의 장르가 Adventure, Animation, Children 등 다양함을 알 수 있습니다.<br>
2번 영화도 세 가지 장르를 가졌습니다. `movie_id`의 값이 고유하지 않다는 점을 알 수 있습니다.<br>

이번에는 데이터가 얼마나 있는지 확인해보겠습니다. 해당 테이블에 대한 행의 갯수는 아래의 SQL 구문으로 확인할 수 있습니다.
````sql
SELECT COUNT(*) FROM 테이블 이름
````


In [63]:
# genres의 행 갯수 가져오기


### (3) `ratings` 테이블 확인하기

`ratings` 데이터를 보면, 사용자를 나타내는 `user_id`와 사용자가 평가한 영화를 나타내는 `movie_id`, 해당 영화에 유저가 부여한 평점인 `rating`, 평점을 남긴 날짜인 `rated_at` 칼럼이 있습니다. 

In [64]:
# ratings 테이블에서 10개 행만 가져오기


이번에는 데이터가 얼마나 있는지 확인해보겠습니다.

In [65]:
# ratings의 행 갯수 가져오기



`genres`와 `movies` 데이터는 Database에서는 상대적으로 데이터 크기가 작은 편에 속합니다. <br>
하지만 `rating` 테이블은 훨씬 더 많은 레코드(2,000만행)가 담겨 있습니다. <br>
이러한 데이터를 가져올 때 위와 같이 코드를 작성하게 되면 동시에 수천만개의 데이터를 요청하게 돼, DBMS에 심한 부하를 가져다 줍니다.<br>
이를 방지해 주기 위해서는 아래와 같이, Chunk 단위로 조금씩 요청한 후 이를 합쳐주는 방식이 보다 안전하고 효율적입니다.<br>

````python
# 방법 1. 한 번에 가져오기
pd.read_sql('SELECT * from ratings', connect) # 금지, 한번에 2000만줄을 동시에 가져옴


# 방법2. chunk 단위로 쪼개서 요청하기. 한번에 10만줄 씩 가져옴
chunks = pd.read_sql('SELECT * from ratings', connect, chunksize=100000)
rating_df = pd.concat([chunk for chunk in chunks])# Chunk 단위로 날라온 Dataframe을 하나로 합쳐주기
````

*caution*

해당 코드는 네트워크 환경에 따라 최대 10분가량의 시간이 소요됩니다. 전체 데이터를 한 번에 처리할 일이 아니라면, 사용하지 않습니다.

# \[ 데이터 탐색하기 \]
---

Pandas는 파이썬의 대표적인 데이터 분석 패키지입니다. 위에서 다루었던 `pd.read_sql` 또한 Pandas의 기능 중 하나로서, Database의 레코드들을 데이터프레임으로 변환하여 파이썬에서 다루기 편하게 만들어줍니다. <br>
Pandas는 '코드로 치는 엑셀'과 비슷하다고 생각하면 됩니다. 코드로서 테이블에서 필요한 정보들을 탐색하고, 계산하고, 시각화하는 기능을 제공해줍니다.<br>

## 1. 각 테이블에 대한 주요 통계정보 확인하기

### (1) `movies` 테이블  탐색하기

다시 `movies` 데이터를 확인해봅시다.

In [66]:
pd.read_sql('SELECT * FROM movies LIMIT 10', connect)

id                        title  release_year
0   1                    Toy Story          1995
1   2                      Jumanji          1995
2   3             Grumpier Old Men          1995
3   4            Waiting to Exhale          1995
4   5  Father of the Bride Part II          1995
5   6                         Heat          1995
6   7                      Sabrina          1995
7   8                 Tom and Huck          1995
8   9                 Sudden Death          1995
9  10                    GoldenEye          1995

일단 몇 년도 영화부터 몇 년도 영화까지 있는지 알아봅시다. 이를 위해서 우선 집계함수를 배워보도록 하겠습니다.<br>
집계 함수에는 여러 가지 존재하고, 대표적으로 아래와 같은 함수들이 있습니다.

| 집계 함수 | 기능 |
| ---- | --- |
| `COUNT(열 이름)` | 데이터의 갯수를 세기 |
| `SUM(열 이름)` | 데이터의 합계를 구하기 |
| `AVG(열 이름)` | 데이터의 평균을 구하기 |
| `MAX(열 이름)` | 데이터의 최댓값 구하기 |
| `MIN(열 이름)` | 데이터의 최솟값 구하기 |

집계함수는 아래와 같은 방식으로 작성하면 됩니다.

````sql
SELECT COUNT(열 이름) FROM 테이블 명
````


In [67]:
# movies 테이블에서 release_year 중 가장 오래된 연도와 가장 최신 연도를 가져오기


1891년도 영화부터 2015년도 영화까지 데이터가 있습니다.
이번에는 출시 연도별로 얼마나 많은 영화의 데이터가 있는지 살펴봅시다. <br>
이를 수행하기 위해서는 Group by 구문을 배워보도록 하겠습니다.<br>
group by 는 어떤 기준을 중심으로 데이터를 합쳐 집계 함수를 적용하고자 할 때 사용됩니다.<br>

````sql
SELECT 집계함수(열 이름) 
FROM 테이블 이름 
GROUP BY 집계 기준 열 이름
````

연도를 집계 기준 열(그룹)로 두고, `id` 칼럼에 `count`함수를 적용하게 되면 해당 그룹의 `id` 갯수를 카운트해줍니다.

In [68]:
# movies 테이블에서 release_year 별 몇개의 영화가 있는지 가져오기


위의 결과를 시각적으로 표시하려면 아래와 같이 작성하면 됩니다.

In [69]:
# Bar 그래프를 이용한 연도 별 영화 갯수 시각화 


그래프를 보면, 1990년대 이후 영화의 평점이 많고, 최근인 2014년도와 2015년도 영화의 평점이 아직 많이 쌓이지 않았음을 알 수 있습니다. 

### (2) `genres` 테이블 탐색하기

`genres` 테이블을 간단히 탐색해보겠습니다. 

In [70]:
pd.read_sql('SELECT * FROM genres LIMIT 10', connect)

movie_id      genre
0         1  Adventure
1         1  Animation
2         1   Children
3         1     Comedy
4         1    Fantasy
5         2  Adventure
6         2   Children
7         2    Fantasy
8         3     Comedy
9         3    Romance

장르별로 얼마나 많은 영화 데이터가 있는지 확인하기 위해서는 아래와 같이 `GROUP BY` 구문을 이용합니다.

In [71]:
# genres 테이블에서 장르별 영화의 갯수를 가져오기


이를 내림차순으로 정렬하고자 한다면, `ORDER BY 정렬 기준 열 이름 DESC`를 이용하면 됩니다.

```` sql
SELECT 열 이름 
FROM 테이블 이름 
ORDER BY 정렬기준 열 이름 [DESC|ASC]
````

In [72]:
# genres 테이블에서 장르별 영화의 갯수를 내림차순으로 가져오기


장르는 총 19가지이고, 장르가 없는 영화는 246개입니다. 드라마 장르의 영화가 가장 많고, IMAX 장르의 영화가 가장 적습니다. 아마도 IMAX가 등장한지 얼마 되지 않았기 때문인 것으로 짐작할 수 있습니다.

In [73]:
# PIE 그래프를 통한 시각화


이번에는 영화별로 몇 개의 장르 정보를 가지고 있는지 살펴봅시다. 

In [74]:
# 영화 별로 몇 개의 장르가 있는지를 세어보기


영화 별 장르의 수를 히스토그램으로 나타내면 아래와 같습니다. 대부분 하나 혹은 두 개의 장르 정보를 가지는 것을 알 수 있습니다. 

In [75]:
# 영화 별 장르 갯수를 histogram으로 세기


### (3) `ratings` 테이블 탐색하기

이번에는 `ratings` 데이터를 확인해봅시다.

In [76]:
pd.read_sql('SELECT * FROM ratings limit 10', connect)

user_id  movie_id  rating    rated_at
0        1         2     3.5  1112486027
1        1        29     3.5  1112484676
2        1        32     3.5  1112484819
3        1        47     3.5  1112484727
4        1        50     3.5  1112484580
5        1       112     3.5  1094785740
6        1       151     4.0  1094785734
7        1       223     4.0  1112485573
8        1       253     4.0  1112484940
9        1       260     4.0  1112484826

여기에서 `rate_at`은 시간을 나타내는 방법 중 하나로, 1970년 1월 1일부터 몇 초가 지났는지를 계산한 결과입니다.

`pd.DataFrame()`으로 데이터를 받아온 후 `rated_at`를 시간값으로 변환하기 위해서는`pd.to_datetime()`을 이용하면 됩니다.

In [77]:
rating_df = pd.read_sql(f'SELECT * FROM ratings LIMIT 10', connect)

rating_df.rated_at = pd.to_datetime(rating_df.rated_at, unit='s')
rating_df.head()

user_id  movie_id  rating            rated_at
0        1         2     3.5 2005-04-02 23:53:47
1        1        29     3.5 2005-04-02 23:31:16
2        1        32     3.5 2005-04-02 23:33:39
3        1        47     3.5 2005-04-02 23:32:07
4        1        50     3.5 2005-04-02 23:29:40

타임스탬프 값을 datetime으로 변경해주면 아래와 같이 시간 계산을 매우 편리하게 수행할 수 있습니다.<br>
혹은 sql 쿼리의 `FROM_UNIXTIME`을 이용하여 바로 변환하여 볼 수도 있습니다.

In [48]:
# ratings 테이블로부터 10개의 행 가져오기
# (FROM_UNIXTIME을 활용하여 rate_at를 바꾸어서)


user_id  movie_id  rating    rated_at FROM_UNIXTIME(rated_at)
0        1         2     3.5  1112486027     2005-04-02 23:53:47
1        1        29     3.5  1112484676     2005-04-02 23:31:16
2        1        32     3.5  1112484819     2005-04-02 23:33:39
3        1        47     3.5  1112484727     2005-04-02 23:32:07
4        1        50     3.5  1112484580     2005-04-02 23:29:40
5        1       112     3.5  1094785740     2004-09-10 03:09:00
6        1       151     4.0  1094785734     2004-09-10 03:08:54
7        1       223     4.0  1112485573     2005-04-02 23:46:13
8        1       253     4.0  1112484940     2005-04-02 23:35:40
9        1       260     4.0  1112484826     2005-04-02 23:33:46

이번에는 연도별로 리뷰 수가 얼마나 되는지 알아봅시다.

In [43]:
# 연도 별 리뷰수를 세기


rated_year  count_ratings
0        1995              4
1        1996        1612609
2        1997         700982
3        1998         308070
4        1999        1198384

유저 별로 평균 선호도가 어떻게 되는지를 계산해 볼 수 있습니다.

In [44]:
# 유저 별 평균 선호도 계산하기


In [ ]:
# 유저 별 평균 선호도를 histogram으로 시각화하기


유저들은 평균적으로 3~4점대의 평점을 많이 남기는 것을 알 수 있습니다. 

# \[ 비개인화 추천 : 아이템의 순위 매기기 \]
---

가장 간단한 형태의 추천은 **잘 나가는 제품 혹은 서비스**을 제시하는 것입니다. <br>
가장 평이 좋은 제품 혹은 가장 구매가 빈번한 제품, 최근에 나온 제품 등을 추천하는 것은 고객에 대한 정보가 전혀 없을 때 매우 유용합니다.<br>
그렇기 때문에 대부분의 커머스 사이트에서 랜딩 페이지에 인기 순위 혹은 신상을 소개하는 경우가 많습니다. <br>
이번에는 `sql`의 정렬 기능`order by`와 `group by` 기능을 이용해 (1) 최신 아이템, (2) 최고 평점 아이템, (3) 최근 인기 아이템 리스트를 만들어보겠습니다.<br>

## 1. 최신(신규) 아이템 추천하기

> 최근에 나온 제품 있나요?

아이템의 출시일(발매일)을 기준으로 정렬을 하여, 가장 최근에 등장한 것들 위주로 가져올 수 있습니다. <br>
영화의 경우 출시시점에 따라 최신 순으로 나열할 수 있을 것입니다. `movies`데이터의 `release_year` 칼럼이 있으므로, 이를 기준으로 상위 10개 아이템을 뽑아보겠습니다. 

In [49]:
# 발매일 기준 최근 10개 가져오기



id               title  release_year
0  129030           The Coven          2015
1  128520  The Wedding Ringer          2015
2  130056  Kingdom of Shadows          2015
3  126482       Strange Magic          2015
4  130578          The Gunman          2015
5  130069           Road Hard          2015
6  130326         Other Girls          2015
7  130840              Spring          2015
8  130073          Cinderella          2015
9  130842       Power/Rangers          2015

위와 같이 10개의 영화가 뽑혔습니다. 하지만 2015년에 출시된 영화는 이 외에도 많이 있죠. <br>
또한 연도만 있지 출시 월일이 없으므로 위의 아이템이 가장 최근에 출시된 영화라고 보기도 어렵습니다.<br>
<br>
이렇게 되면 최신 아이템 중에서 어떤 것은 노출되고 어떤 것은 노출되지 않는다는 단점이 있습니다. <br>
이런 경우, 간단히 아래와 같이 무작위로 최근에 등장한 아이템중에서 무작위로 선정하여 노출되는 아이템을 다양하게 해줍니다.<br>

In [78]:
# 발매일 기준 최근 10개 무작위로 가져오기



앞서 사용한 쿼리에서 `WHERE`절을 추가하여 최근 출시 연도를 `release_year`의 조건으로 걸어줘야 합니다. 
```sql
SELECT * 
FROM movies 
WHERE release_year = (최근 출시 연도)
ORDER BY RAND()
LIMIT 10
```

최근 출시 연도는 어떻게 알 수 있을까요? 아래의 쿼리로 알 수 있습니다. 
```sql
SELECT MAX(release_year) 
FROM movies
```

위의 쿼리를 앞서 만든 쿼리의 `최근 출시 연도`부분에 넣어주면 됩니다. 이렇게 쿼리 안에 쿼리를 넣어 만드는 쿼리문을 "서브 쿼리"라고 합니다.

## 2. 최고 평점 아이템 추천하기
> 평이 제일 좋은 물건은 무엇인가요?

평균 평점을 이용해 평이 제일 좋은 물건을 뽑아서 보여줄 수도 있을 것입니다.<br> 
다만, 평점이 높은 아이템을 뽑을 때는 한개, 두개의 평만 있는 것들은 제외시킬 필요가 있습니다.<br> 
충분히 많은 사람들로부터 검증받지 못한 것들은 그 평을 신뢰할 수 없기 때문입니다. <br>
보통 응답조사를 할 때 300명 이상을 유의미하다고 판단하기 때문에, 300명 이상 평가를 받은 영화들만 추려내도록 하겠습니다. <br>

먼저, 영화별 평균 평점과 리뷰의 수를 구해야 할 것입니다. 이를 구하는 쿼리문은 아래와 같습니다.<br>

````sql 
SELECT movie_id, 
AVG(rating) as mean_rating, 
COUNT(user_id) as count_rating
FROM ratings
GROUP BY movie_id;

````
위의 코드로도 값을 구할 수 있지만 위와 같이 전체 데이터셋을 기준으로 한 번에 통계치를 집계하는 경우에는 대개 많은 시간을 소요합니다. <br>
그렇기 때문에 주로 실무에서는 이런 작업은 미리 통계 테이블을 구성한 후, 통계 테이블에서 정보를 가져오는 식으로 시스템을 구축합니다. <br>
통계 테이블은 서비스의 형태에 따라 짧게는 분단위, 길게는 일 단위로 정보를 갱신합니다.<br>
<br>
`movielens` 데이터베이스에 `rating_stats`테이블을 구성해두었습니다.

In [57]:
# ratings_stats 가져오기
pd.read_sql('''
SELECT *
FROM rating_stats
''', connect)

movie_id  rating_mean  rating_count
0             1      3.92124         49695
1             2      3.21198         22243
2             3      3.15104         12735
3             4      2.86139          2756
4             5      3.06459         12161
...         ...          ...           ...
26739    131254      4.00000             1
26740    131256      4.00000             1
26741    131258      2.50000             1
26742    131260      3.00000             1
26743    131262      4.00000             1

[26744 rows x 3 columns]

300개 이상 평점을 받은 영화 중 Top 10을 선정하도록 하겠습니다.

In [79]:
# 300개 이상의 평점 정보가 존재하는 영화 중 rating_mean 기준 Top 10을 선정



결과를 보면, 다양한 분야의 영화가 인기 영화로 리스트업 된 것 같습니다. 다만 전체에서 평점이 높은 영화를 뽑다보니 다소 무거운 내용의 영화가 많이 뽑힌듯 합니다. <br>
장르 별로 최고 평점 영화들을 뽑아서 보여준다면, 특정 장르에 관심이 많은 유저에게 유용한 정보를 제공할 수 있을 것입니다. 이번에는 코미디 영화만 추려서 동일한 방법으로 최고 평점 영화를 뽑아봅시다. 

In [81]:
# 300개 이상의 평점 정보가 존재하는 'Comedy' 영화 중 rating_mean 기준 Top 10을 선정



앞서 사용한 쿼리에서 `WHERE`절에 `AND` 이하에 `movie_id IN (코미디 영화의 movie_id)`라는 조건을 추가해야 합니다. 
```sql
SELECT *
FROM rating_stats
WHERE rating_count >= 300 AND
movie_id IN (코미디 영화의 movie_id)
ORDER BY rating_mean DESC
LIMIT 10
```

코미디 영화의 `movie_id`는 어떻게 알 수 있을까요? 코메디 영화의 `movie_id`는 아래의 쿼리로 알 수 있습니다. 
```sql
SELECT movie_id
FROM genres
WHERE genre = 'Comedy'
```

위의 쿼리를 앞서 만든 쿼리의 `코미디 영화의 movie_id`부분에 넣어주면 됩니다. 

## 3. 최근 인기 아이템(HOT) 추천하기

> 요즘 사람들이 제일 많이 찾는 것이 무엇인가요?

영화의 평점이 높은 순으로 영화를 추천하게 되면 어떤 약점이 있을까요? 오래된 영화일수록 평가가 많이 쌓이게 되기 때문에, 과거 평가 기준으로 영화를 추천하게 됩니다. <br> 
최근에 개봉한 영화는 아직 평점이 많지 않아 추천되지 않을 수 있습니다. 최신 영화를 위주로 추천하고 싶다면, 아래와 같은 방식으로 최근 개봉영화만을 추려내어 
추천을 해야 합니다. <br>

가장 마지막 출시연도로부터 4년 이내의 영화를 뽑아봅시다.

In [82]:
# movies로부터 최근 4년 간 출시되었던 영화 정보 가져오기


위 조건에 해당하는 영화 중에서 가장 높은 평점을 가진 영화를 추천하기 위해서는 어떻게 코드를 작성해야 할까요? 

```sql
SELECT *
FROM rating_stats
WHERE rating_count >= 300 AND
movie_id IN (최근 4년간 출시된 영화의 movie_id)
ORDER BY rating_mean DESC
LIMIT 10
```

최근 4년간 출시된 영화의 `movie_id`는 어떻게 알 수 있을까요? 앞서 쿼리한 대로 아래와 같이 코드를 작성할 수 있습니다.
```sql
SELECT *
FROM movies
WHERE release_year >= (
    SELECT max(release_year)-3 
    FROM movies) 
```

위의 쿼리를 앞서 만든 쿼리의 `최근 4년간 출시된 영화의 movie_id`부분에 넣어주면 됩니다. 

In [83]:
# 최근 4년간 영화 중에서 rating_stats에서 가장 평점이 좋은 영화 가져오기


`top10_recent_movies`영화를 기준으로 앞서 인기 영화를 뽑은 기준을 그대로 적용하여 최근 인기 아이템을 뽑아보겠습니다. <br>
결과를 확인해보면, 앞서 전체 데이터에서 인기 영화를 뽑았을 때에는 보이지 않았던 최신 영화가 리스트됨을 알 수 있습니다. 

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2020/01/13


---